In [1]:
# This will clone and install the branch that has teh current changes on it. If you want to see the output of the install to be sure it 
# was successfull, remove the > /dev/null 2>&1 in the second line.  

#!git clone --branch 159-change-calculation-of-default-grid-parameters https://github.com/casangi/astrohack.git
#!pip install -e astrohack/ > /dev/null 2>&1
!pip install -e .. > /dev/null 2>&1

In [3]:
import os
import json
import astrohack

import numpy as np

In [4]:
import distributed

from astrohack.client import astrohack_local_client

DEFAULT_DASK_ADDRESS="tcp://localhost:8786"

log_parms = {
    'log_level':'DEBUG'
}

if not distributed.client._get_global_client():
    try:
        client = distributed.Client(DEFAULT_DASK_ADDRESS, timeout=2)
        
    except OSError:
        os.environ["DASK_SCHEDULER_ADDRESS"] = DEFAULT_DASK_ADDRESS
        client=astrohack_local_client(
            cores=2,
            memory_limit='8GB',
            log_parms=log_parms
        )
        
else:
    client = distributed.client._get_global_client()

client

2023-10-06 11:29:12,856 - INFO     - astrohack - (_check_parms.py:135) - [_check_logger_parms]: Setting default log_to_term to True.
2023-10-06 11:29:12,857 - INFO     - astrohack - (_check_parms.py:135) - [_check_logger_parms]: Setting default log_to_file to False.
2023-10-06 11:29:12,857 - INFO     - astrohack - (_check_parms.py:135) - [_check_logger_parms]: Setting default log_file to hack_.
2023-10-06 11:29:12,858 - INFO     - astrohack - (_check_parms.py:135) - [_check_worker_logger_parms]: Setting default log_to_term to False.
2023-10-06 11:29:12,858 - INFO     - astrohack - (_check_parms.py:135) - [_check_worker_logger_parms]: Setting default log_to_file to False.
2023-10-06 11:29:12,858 - INFO     - astrohack - (_check_parms.py:135) - [_check_worker_logger_parms]: Setting default log_file to hack_.
2023-10-06 11:29:12,859 - INFO     - astrohack - (_check_parms.py:135) - [_check_worker_logger_parms]: Setting default log_level to INFO.
2023-10-06 11:29:14,741 - INFO     - astroha

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 2
Total threads: 2,Total memory: 14.90 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:44093,Workers: 2
Dashboard: http://127.0.0.1:8787/status,Total threads: 2
Started: Just now,Total memory: 14.90 GiB
Comm: tcp://127.0.0.1:36255,Total threads: 1
Dashboard: http://127.0.0.1:33257/status,Memory: 7.45 GiB
Nanny: tcp://127.0.0.1:44471,


In [5]:
ms_file = "/lustre/cv/users/jhoskins/otf33.ms"

In [6]:
from astrohack.extract_holog import generate_holog_obs_dict

holog_obs_dict = generate_holog_obs_dict(
    ms_name=ms_file,
    point_name="data/otf33.point.zarr",
)

2023-10-06 11:29:16,689 - INFO     - astrohack - (_extract_holog.py:432) - Writing distance matrix to /export/home/fornax/jhoskins/Development/validation/astrohack-unit-test-docstrings/stakeholder_test/.baseline_distance_matrix.csv ...


In [7]:
from astrohack.extract_pointing import extract_pointing

point_mds = extract_pointing(
      ms_name=ms_file,
      point_name="data/otf33.point.zarr",
      parallel=True,
      overwrite=True
)

2023-10-06 11:29:16,718 - WARNING  - astrohack - (_dio.py:43) - [extract_pointing]: data/otf33.point.zarr will be overwritten.
Successful readonly open of usernoread-locked table /lustre/cv/users/jhoskins/otf33.ms/ANTENNA: 8 columns, 24 rows
2023-10-06 11:29:32,199 - INFO     - astrohack - (extract_pointing.py:129) - [extract_pointing]: Finished processing


In [8]:
# Here I reduced the size of the holog_obs_dict for quicker testing. You can comment this out of edit it as needed depending on your testing goals.

from astrohack.extract_holog import generate_holog_obs_dict

holog_obs_dict = generate_holog_obs_dict(
    ms_name=ms_file,
    point_name="data/otf33.point.zarr",
)

2023-10-06 11:29:34,020 - INFO     - astrohack - (_extract_holog.py:432) - Writing distance matrix to /export/home/fornax/jhoskins/Development/validation/astrohack-unit-test-docstrings/stakeholder_test/.baseline_distance_matrix.csv ...


### The holog observations dictionary has now been turned into an extended dictionary object similar to the `holog_mds`. There are a couple of new functionalities available to the user now:

- The `select` function allows the user to trim the holog_obs_dict on all major axes (ddi, map, antenna, scan, baseline) in a much simpler way than previsouly.
- The `print` function allows the user to inspect the holog_obs_dict with both a static, ascii print out style and a dynamic, collapsible json style.
- A static method is available to allow the user to select the `n` closest baseline associated with a given mapping antenna adn return them in a list that cane be used with the baseline key option allowing the user to trim the holog_obs_dict based on baseline.

### Each of these options is demonstrated below.

In [9]:
#get_nearest_baselines(antenna="ea25", n_baselines=5)

In [10]:
# There are two options here for style, static and dynamic. Static returns a formatted ascii output while dynamic returns a dynamic json object.
holog_obs_dict.print(style="dynamic")

<IPython.core.display.JSON object>

### `Select(key, value, inplace, **kwargs)`
### The select function allows the user to trim the `holog_obs_dict`. The `key` (ddi, map, antenna, scan, baseline) input gives the dictionary key you would like to select and `value` is the value you want to keep. Everything else is deleted. There is also an `inplace` boolean input. This specifies whether you want to modify the calling dictionay or return a new dictionary. The additional arguments, `kwarags` is currently only useful for the baseline example explain further down.

### One key feature is also that all select calls are chainable. This allows for selecting on mutiple keys in a simple modular way. An example is shown below.

In [11]:
# Select only ddi=0 and return a new dictionary, from the original dictionary above.
#trimmed_dict = holog_obs_dict.select(key="ddi", value=1, inplace=False)
#trimmed_dict.print(style="dynamic")

In [12]:
# Here we will select only following (key,value) pairs: 
#
# ddi=0
# map=0
# scans=6, 7, 8, 9, 10
# antenna='ea22', 'ea15', 'ea25'
#
# We do this using function chaining. Because each call returns a new object, another call can be directly invoked.

trimmed_dict = holog_obs_dict.select(
    key="ddi", 
    value=0, 
    inplace=False
).select(
    key="map", 
    value=0, 
    inplace=False
).select(
    key="scan", 
    value=[6, 7, 8, 9, 10], 
    inplace=False
).select(
    key="antenna", 
    value=['ea22', 'ea15', 'ea25'], 
    inplace=False
)

trimmed_dict.print(style="dynamic")

<IPython.core.display.JSON object>

### `Select(...)` with the baseline option:

### In order to trim the dictionary based on the shortest baseline formed using a given mapping antenna we require the use of two functions. First we use the static method to get the reference antennas that for the `n` shortest baselines given and antenna.

In [13]:
# Since it doesn't mdoify the dictionary object, this can be called on the new returned dictionary or the original dictionary. The following 
# gives the antennas for the three shortest baselines formed with the mapping antenna, 'ea25'. Note these don't take into account whether the 
# antenna is reference or mapping, it only calculates the baseline lengths for reference.
nearest_baselines = holog_obs_dict.get_nearest_baselines(antenna="ea15", n_baselines=24)
nearest_baselines

['ea23',
 'ea22',
 'ea28',
 'ea12',
 'ea13',
 'ea18',
 'ea09',
 'ea16',
 'ea01',
 'ea19',
 'ea14',
 'ea27',
 'ea08',
 'ea26',
 'ea04',
 'ea17',
 'ea11',
 'ea25',
 'ea02',
 'ea20',
 'ea03',
 'ea24',
 'ea21']

In [22]:
# With the reference antenna list in-hand we can now trim the dictionary using the select method as follows
trimmed_dict = {}

trimmed_dict = holog_obs_dict.select(
    key="ddi", 
    value=1,
    inplace=False
).select(
    key="scan", 
    value=[
        6, 7, 8, 9, 10, 11, 12, 13, 14, 15,
        16, 17, 19, 20, 21, 22, 23, 24, 25, 
        26, 27, 28, 29, 30, 35, 36, 37, 38, 
        39, 40
    ],
    inplace=False
).select(
    key="baseline", 
    value="ea15", 
    n_baselines=3, 
    inplace=False
)

trimmed_dict.print(style="dynamic")

<IPython.core.display.JSON object>

In [ ]:
# You will notice that there is a warning thrown below regarding the mutliple values of cell size and pixel number found in the pointing file
# but suggested values are computed. They are NOT guarenteed to work though. For instance, below you will see the number of pixels is very small
# and if you use the resonably calculated pixle value suggested, you will get a singular matrix. There is still work to be done.

from astrohack.extract_holog import extract_holog

holog_mds = extract_holog(
    ms_name=ms_file,
    point_name="data/otf33.point.zarr",
    holog_name="data/otf33.holog.zarr",
    holog_obs_dict=trimmed_dict,
    data_column='CORRECTED_DATA',
    parallel=True,
    overwrite=True
)

In [ ]:
with open("data/otf33.holog.zarr/.holog_attr") as json_file:
    json_object = json.load(json_file)
    
print("Cell size: ", json_object["cell_size"])
print("Number of pixels: ", json_object["n_pix"])

In [ ]:
from astrohack.holog import holog

image_mds = holog(
    holog_name="data/otf33.holog.zarr",
    grid_size=np.array([30, 30]),
    overwrite=True,
    parallel=True
)

In [ ]:
#
# A list of colormaps available in matplotlib can be found here:
#
# https://matplotlib.org/stable/users/explain/colors/colormaps.html
#

image_mds.plot_apertures(
    destination="plots", 
    ant_id="ea25",
    colormap="RdYlGn",
    display=True
)